<a href="https://colab.research.google.com/github/shunrei9841-sudo/Guadalupe/blob/main/Parcial%202-8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import math

# --- Parámetros de la simulación ---
N = 1000000     # Número total de muestras
I_analytic = math.pi / 4 # Valor de referencia
print(f"Valor Analítico de la integral I: {I_analytic:.6f}")
print("-" * 50)

# 1. Simulación Simple (Base para la Varianza)
U_simple = np.random.uniform(0, 1, N)
g_U_simple = np.sqrt(1 - U_simple**2)
Var_g_U_simple = np.var(g_U_simple, ddof=1)
Var_I_simple = Var_g_U_simple / N
print("--- 1. Simulación Simple (Referencia) ---")
print(f"Varianza del Estimador (Simple): {Var_I_simple:.12f}")


# --- A) Muestreo Estratificado ---
print("\n--- A) Muestreo Estratificado ---")

# Usaremos K = 10 estratos
K = 10
n_k = N // K # Número de muestras por estrato (n_k = N/K)
Var_g_k = np.zeros(K)
I_hat_k = np.zeros(K)

for k in range(K):
    # Generar muestras U_k en el k-ésimo estrato [ (k)/K, (k+1)/K ]
    # U_k ~ Unif(k/K, (k+1)/K)
    U_k = np.random.uniform(k/K, (k+1)/K, n_k)

    # Calcular la función g(U_k) = sqrt(1 - U_k^2)
    g_U_k = np.sqrt(1 - U_k**2)

    # Estimador I_hat_k para el estrato k (Media * ancho del estrato 1/K)
    I_hat_k[k] = np.mean(g_U_k) / K

    # Varianza dentro del estrato k (para el cálculo de la varianza total)
    Var_g_k[k] = np.var(g_U_k, ddof=1)

# Estimación de la integral I
I_hat_estratificado = np.sum(I_hat_k)

# Varianza del Estimador Estratificado
# Var[I_hat] = Sum_{k=1}^K ( (1/K)^2 * Var[g(U_k)] / n_k )
# Var[g(U_k)] / n_k = Varianza_muestral_del_estrato_k
Var_I_estratificado = np.sum((1/K)**2 * Var_g_k / n_k)

# Porcentaje de reducción de varianza
Reduccion_estratificado = (Var_I_simple - Var_I_estratificado) / Var_I_simple * 100

print(f"Estimación I (Estratificado, K={K}): {I_hat_estratificado:.6f}")
print(f"Varianza del Estimador (Estratificado): {Var_I_estratificado:.12f}")
print(f"Reducción de Varianza: {Reduccion_estratificado:.2f}%")
print("-" * 50)


# --- B) Combinación de Técnicas ---
print("--- B) Combinación de Técnicas (Antitéticas + Control + Estratificación) ---")

# 1. Estratificación (K=10)
# 2. Variables Antitéticas (U_anti = 1 - U_k)
# 3. Variable de Control (C = U_k). E[C_k] = (k + 0.5) / K

Var_I_comb = 0
I_hat_comb = 0

# La variable de control C = U (o U_k en el estrato) tiene un valor esperado conocido:
# E[U] = 0.5 para U ~ Unif(0, 1)
# E[U_k] = ( (k+1)/K + k/K ) / 2 = (2k + 1) / (2K)
E_U_k_analytic = (2 * np.arange(K) + 1) / (2 * K)
Var_U_k_analytic = (1/K)**2 / 12

for k in range(K):
    # Generar n_k/2 muestras uniformes U_k en el estrato
    n_half = n_k // 2
    U_k = np.random.uniform(k/K, (k+1)/K, n_half)

    # 1. Antitéticas: U_k_anti
    U_k_anti = (k/K + (k+1)/K) - U_k # U_k_anti = limite_inferior + limite_superior - U_k

    # 2. Función de interés y su antitética: g(U) = sqrt(1 - U^2)
    g_U_k = np.sqrt(1 - U_k**2)
    g_U_k_anti = np.sqrt(1 - U_k_anti**2)

    # 3. Variable de Control C_k = U_k
    C_k = np.concatenate((U_k, U_k_anti)) # Muestras combinadas de U en el estrato
    g_k_combined = np.concatenate((g_U_k, g_U_k_anti)) # Muestras combinadas de g(U) en el estrato

    # Parámetros analíticos del estrato k
    E_C_k = E_U_k_analytic[k]

    # Calcular b*_k óptimo (usando valores muestrales del estrato)
    # b*_k = Cov[g_k_combined, C_k] / Var[C_k]
    Cov_matrix_k = np.cov(g_k_combined, C_k)
    Cov_gC_k = Cov_matrix_k[0, 1]
    Var_C_k_sample = Cov_matrix_k[1, 1]

    # Evitar división por cero si la varianza es cero (debe ser muy raro)
    b_optimo_k = Cov_gC_k / Var_C_k_sample if Var_C_k_sample > 1e-10 else 0

    # 4. Estimador Combinado para el estrato k
    # g_ctrl_k = g_k_combined - b*_k * (C_k - E[C_k])
    g_ctrl_k = g_k_combined - b_optimo_k * (C_k - E_C_k)

    # 5. Contribución a la estimación total: Media de g_ctrl_k * (ancho 1/K)
    I_hat_comb += np.mean(g_ctrl_k) / K

    # 6. Contribución a la varianza total: (1/K)^2 * Var[g_ctrl_k] / N_k
    Var_g_ctrl_k = np.var(g_ctrl_k, ddof=1)
    Var_I_comb += (1/K)**2 * Var_g_ctrl_k / n_k


# Porcentaje de reducción de varianza
Reduccion_comb = (Var_I_simple - Var_I_comb) / Var_I_simple * 100

print(f"Estimación I (Combinada): {I_hat_comb:.6f}")
print(f"Varianza del Estimador (Combinada): {Var_I_comb:.12f}")
print(f"Reducción de Varianza: {Reduccion_comb:.2f}%")

Valor Analítico de la integral I: 0.785398
--------------------------------------------------
--- 1. Simulación Simple (Referencia) ---
Varianza del Estimador (Simple): 0.000000049837

--- A) Muestreo Estratificado ---
Estimación I (Estratificado, K=10): 0.785413
Varianza del Estimador (Estratificado): 0.000000001514
Reducción de Varianza: 96.96%
--------------------------------------------------
--- B) Combinación de Técnicas (Antitéticas + Control + Estratificación) ---
Estimación I (Combinada): 0.785411
Varianza del Estimador (Combinada): 0.000000000047
Reducción de Varianza: 99.90%
